In [79]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from Laser import Laser
import datetime

# Import custom functions

In [ ]:
# read G-Code file and return a list of strings
def read_gcode_file(filename):
    lst = []
    with open(filename, 'r') as f:
        gcode = f.readlines()
    for line in gcode:
        lst.append(line)
    
    return lst
    

In [80]:
def findAngle(x,y,x_last,y_last):
    newAngle = np.arctan2((y-y_last),(x-x_last))
    if newAngle < 0:
        newAngle = 2*np.pi - np.abs(newAngle)
    # convert to degrees
    newAngle = newAngle*180/np.pi
    return newAngle

In [81]:
def find_closest_angle(target_angle, anglesObj):
    angles = anglesObj.angles
    # Convert all angles to be in the range [0, 360)
    normalized_angles = np.mod(angles, 360)
    target_angle = target_angle % 360

    # Calculate the distance between the target angle and each angle
    distances = np.minimum(np.abs(target_angle - normalized_angles), 360 - np.abs(target_angle - normalized_angles))
    
    # Find the index of the smallest distance
    closest_index = np.argmin(distances)

    # Calculate the delta between the target angle and the closest angle
    delta = normalized_angles[closest_index] - target_angle

    n = angles.shape[0]

   
#######
    if delta > 180:
        delta -= 360
    elif delta < -180:
        delta += 360
#######

    if (delta > ((180/n)+0.5)):
        print(f'ERROR in delta calculation! delta: {delta}; index {closest_index}; normalizedAbgle {normalized_angles[closest_index]}')



    # update angles
    anglesObj.update_angles(delta)    

    return closest_index, delta, target_angle

In [82]:
# def function to loop thru array of coords
def loopCoords(coords, laserObj):
    deltaErr = 0
    # split coords
    X = [x[0] for x in coords]
    Y = [x[1] for x in coords]
    Z = [x[2] for x in coords]
    
    angles = []

    for i in range(len(X)-1):
        x = X[i]
        x_last = X[i-1]
        y = Y[i]
        y_last = Y[i-1]


        angle = findAngle(x,y,x_last,y_last)
        angles.append(angle)

    angles = np.array(angles)

    # find closest angle
    closest_angles = []
    deltas = []
    target_angles = []
    for i in range(len(angles)):
        closestIndex, delta, target_angle = find_closest_angle(angles[i], laserObj)
        closest_angles.append(closestIndex)
        deltas.append(delta)
        target_angles.append(target_angle)
        # collect errors
        if delta > (360/laserObj.angles.shape[0]):
            deltaErr += 1

    # save all data in a csv file: X, Y, Z, angles, closestIndex, deltas, deltaErr
    # convert to numpy arrays
    X = np.array(X)
    Y = np.array(Y)
    Z = np.array(Z)
    angles = np.array(angles)
    closestIndex = np.array(closest_angles)
    deltas = np.array(deltas)
    deltaErr = np.array(deltaErr)
    target_angles = np.array(target_angles)

    

    # remove first element of each array
    X = np.delete(X, 0)
    Y = np.delete(Y, 0)
    Z = np.delete(Z, 0)


    # print dimensions of X, Y, Z, angles, closestIndex, deltas, deltaErr
    print(f'X: {X.shape}; Y: {Y.shape}; Z: {Z.shape}; angles: {angles.shape}; closestIndex: {closestIndex.shape}; deltas: {deltas.shape}')

    np.savetxt('dataP5.csv', np.column_stack((X,Y,Z,angles,closestIndex,deltas, target_angles)), delimiter=',', fmt='%s')

    return angles, closestIndex, deltas, deltaErr
   

In [83]:
# load test data
p = np.load('npOutput.npy')
print(p.shape)

(9828, 3)


In [84]:
# init lasers
nLasers = 4
Laser = Laser(nLasers)

# get coords
coords = np.array(p)

# run thru loop
angles, closest_angles, deltas, error = loopCoords(coords, Laser)

#print(f'Errors in delta calculation: {error}; Total laser movements: {Laser.absLaserMovement}')

X: (9827,); Y: (9827,); Z: (9827,); angles: (9827,); closestIndex: (9827,); deltas: (9827,)


In [85]:
# safe all data as npy in the LogFiles folder, but create a new subfolder for each run, foldernames are an incrementing number
# check highest number in LogFiles folder


# create new folder
newFolder = 'LogFiles/' + str(datetime.datetime.now())
print(newFolder)
os.mkdir(newFolder)

# save data
np.save(newFolder + '/angles.npy', angles)
np.save(newFolder + '/closest_angles.npy', closest_angles)
np.save(newFolder + '/deltas.npy', deltas)

# save a csv file with following data: x, y, z, angle, closest_angle, delta


# save error, total laser movement and nLasers as json file
with open(newFolder + '/Info.json', 'w') as f:
    f.write('{\n')
    f.write(f'\t"error": {error},\n')
    f.write(f'\t"totalLaserMovement": {Laser.absLaserMovement},\n')
    f.write(f'\t"nLasers": {nLasers}\n')
    f.write('}')

print(f'Error abs: {error}; Error rel: {error/np.shape(coords)[0]}')

LogFiles/2023-03-06 10:03:36.377990
Error abs: 0; Error rel: 0.0


In [86]:
# safe the parameters into a csv file for later comparison
# open log file
logFile = open('logFile.csv', 'a')
# write header only if file is empty
if os.stat('logFile.csv').st_size == 0:
    logFile.write('Datetime;absLaserMovement;nLaser\n')

# write data on new line
logFile.write(f'{datetime.datetime.now()};{Laser.absLaserMovement};{nLasers}\n')



# close log file
logFile.close()


